In [1]:
# !pip install ortools

In [36]:
import pandas as pd
import numpy as np
import itertools
import time

from ortools.linear_solver import pywraplp
from itertools import repeat
%matplotlib inline

In [37]:
#Number of w is N=8 and s = m*m = 22*22 = 484 is 492
# intialise data of lists.

data = {'Asset 1':[  7.5,   8.4,  6.1,  5.2,  5.5,
                     7.7,  10.9, 12.7, 15.6, 11.7,
                     9.2,  10.3,    8,  6.3,  6.1,
                     7.1,   8.7,    8,  5.7,  3.6,  3.1,  4.5],
        'Asset 2':[ -5.8,     2,  5.6, 17.5,  0.2, 
                    -1.8,  -2.2, -5.3,  0.3, 46.5,
                    -1.5,  15.9, 36.6, 30.9, -7.5,
                     8.6,  21.2,  5.4, 19.3,  7.9, 21.7,-11.1],
        'Asset 3':[-14.8, -26.5, 37.1, 23.6, -7.4,
                     6.4,  18.4, 32.3, -5.1, 21.5,
                    22.4,   6.1, 31.6, 18.6,  5.2,
                    16.5,  31.6, -3.2, 30.4,  7.6,   10,  1.2],
        'Asset 4':[-18.5, -28.4, 38.5, 26.6, -2.6,
                     9.3,  25.6, 33.7, -3.7, 18.7,
                    23.5,     3, 32.6, 16.1,  2.3,
                    17.9,  29.2, -6.2, 34.2,    9, 11.3, -0.1],
        'Asset 5':[-30.2, -33.8, 31.8,   28,  9.3, 
                    14.6,  30.7, 36.7,   -1, 21.3,
                    21.7,  -9.7, 33.3,  8.6, -4.1,
                    16.5,  20.4,  -17, 59.4, 17.4, 16.2, -3.2],
        'Asset 6':[  2.3,   0.2, 12.3, 15.6,    3,  
                     1.2,   2.3,  3.1,  7.3, 31.1,
                       8,    15, 21.3, 15.6,  2.3,
                     7.6,  14.2,  8.3, 16.1,  7.6,   11, -3.5],
        'Asset 7':[-14.9, -23.2, 35.4,  2.5, 18.1, 
                    32.6,   4.8, 22.6, -2.3, -1.9,
                    23.7,   7.4, 56.2, 69.4, 24.6,
                    28.3,  10.5,-23.4, 12.1,-12.2, 32.6,  7.8],
        'Asset 8':[ 67.7,  72.2,  -24,   -4,   20, 
                    29.5,  21.2, 29.6,-31.2,  8.4,
                   -12.8, -17.5,  0.6, 21.6, 24.4,
                   -13.9,  -2.3, -7.8, -4.2, -7.4, 14.6,   -1]}

# Create DataFrame
df = pd.DataFrame(data)

In [38]:
# Number of assets
NumA = len(df.columns)
# Number of states
NumS = len(df)

# List for col index
cols = [f'Asset {i}' for i in range(1, NumA+1)]

# Create an array of weights, each set to 1/NumA
weight = np.full(NumA, 1/NumA)

# Calculate the weighted average of the columns in the dataframe
y_k = np.average(np.asarray(df[cols]), weights=weight, axis=1)

# Calculate the probability of each state 
p_k = 1/NumS

v_i = np.array([])

for x in range(NumS):
    yp = y_k[x]-y_k
    yp[yp < 0] = 0
    v_i = np.append(v_i, np.mean(yp))


In [39]:
# Instantiate a Glop solver and naming it.
solver = pywraplp.Solver.CreateSolver('SCIP')


# Declare arrays to hold our variables.
# The below code will store information to objective "solver". 
# Make sure that you didn't store the same information repeatly in the "solver"
weight_A = [solver.NumVar(0.0, solver.infinity(), f'w_{i+1}') for i in range(NumA)]
s_ik = [solver.NumVar(0.0, solver.infinity(), f's_{i+1}_{k+1}') for i in range(NumS) for k in range(NumS)]

print('Number of variables =', solver.NumVariables())

#First inequality constraint
for i in range(NumS):
    for k in range(NumS):
        solver.Add(sum(df[cols[n]][k]*weight_A[n] for n in range(NumA)) + s_ik[i*NumS+k] >= y_k[i])

#Second inequality constraint
for i in range(NumS):
    solver.Add(sum(p_k*s_ik[i*NumS+k] for k in range(NumS)  ) <= v_i[i])
    

solver.Add(sum(weight_A[n] for n in range(NumA) ) == 1)

print('Number of constraints =', solver.NumConstraints())

Number of variables = 492
Number of constraints = 507


In [40]:
solver.Maximize(sum(sum(p_k*df[cols[n]][k]*weight_A[n] for n in range(NumA)) for k in range(NumS)))

In [41]:
# Solve the system.
status = solver.Solve()

In [42]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for n in range(NumA):
        print(f'w_{n+1} =', weight_A[n].solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 11.008198995664435
w_1 = 0.0
w_2 = 0.0
w_3 = 0.06803595208792393
w_4 = 0.18800344323438142
w_5 = 0.0
w_6 = 0.3913755414860866
w_7 = 0.23092417350675995
w_8 = 0.12166088968484806
